In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import csv
import requests
import json
import re

In [ ]:
def get_blog_url(soup):
    div_ = soup.find_all('div', attrs={'class': 'FL PR20'})
    url_list = []
    for title in div_:
        href=title.find('a')['href']
        url_list.append("https://www.moneycontrol.com/"+href)
    return url_list

In [ ]:
def get_blog_content(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')
    all_scripts = soup.find_all('script', attrs= {'type':'application/ld+json'})
    raw_article_str = all_scripts[2].get_text().replace('\r\n', ' ')
    parts = re.split(r"""("[^"]*"|'[^']*')""", raw_article_str)
    parts[::2] = map(lambda s: "".join(s.split()), parts[::2])
    article_str = "".join(parts)
    article_str = article_str[1:]
    article_str = article_str[:-1]
    article_dict = json.loads(article_str)
    return article_dict


# def get_blog_content(url):
#     request = requests.get(url)
#     soup = BeautifulSoup(request.text, 'html.parser')
#     all_scripts = soup.find_all('script', attrs= {'type':'application/ld+json'})
#     raw_article_str = all_scripts[2].get_text().replace('\r\n', ' ')
#     parts = re.split(r"""("[^"]*"|'[^']*')""", raw_article_str)
#     parts[::2] = map(lambda s: "".join(s.split()), parts[::2])
#     article_str = "".join(parts)
#     article_str = article_str[1:]
#     article_str = article_str[:-1]
#     article_dict = json.loads(article_str)
#     all_tags = soup.find_all('div', attrs={'class':'tags_first_line'})
#     lst_all_tags = []
#     for i in all_tags:
#         lst_all_tags.append(i.get_text())
#     tags = lst_all_tags[0].replace('TAGS:', '')
#     tags = tags.replace('\n', '')
#     tags = tags.split('#')
#     tags = tags[1:]
#     tags = ', '.join([str(elem).strip() for elem in tags])
#     article_dict['tags'] = tags
#     return article_dict

In [ ]:
# def get_page_no(url, sc_id, page_no, next, year):
#     request = requests.get(url)
#     soup = BeautifulSoup(request.text, 'html.parser')
#     all_page_no = soup.find_all('div', attrs={'class':'pages MR10 MT15'})
#     page_list = [i.text for i in all_page_no[0].find_all('a')]
#     if any(map(str.isdigit, page_list[-1])):
#         return int(page_list[-1]), next
#     else:
#         next = next + 1
#         page_no = int(page_list[-2])
#         url = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id="+sc_id+"&scat=&pageno="+str(page_no)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
#         return get_page_no(url, sc_id, page_no, next, year)


# def get_page_no(url, sc_id, page_no, next, year):
#     request = requests.get(url)
#     soup = BeautifulSoup(request.text, 'html.parser')
#     all_page_no = soup.find_all('div', attrs={'class': 'pages MR10 MT15'})
    
#     # Check if 'all_page_no' is empty or not found
#     if not all_page_no:
#         print("Pagination not found on the page.")
#         return 0, next

#     page_list = [i.text for i in all_page_no[0].find_all('a')]
    
#     # Check if 'page_list' is empty or not found
#     if not page_list:
#         print("Page numbers not found in pagination.")
#         return 0, next

#     if any(map(str.isdigit, page_list[-1])):
#         return int(page_list[-1]), next
#     else:
#         next = next + 1
#         page_no = int(page_list[-2])
#         url = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id=" + sc_id + "&scat=&pageno=" + str(
#             page_no) + "&next=" + str(next) + "&durationType=Y&Year=" + str(year) + "&duration=1&news_type="
#         return get_page_no(url, sc_id, page_no, next, year)

def get_page_no(url, sc_id, page_no, next, year):
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')
    all_page_no = soup.find_all('div', attrs={'class': 'pages MR10 MT15'})
    
    # Check if 'all_page_no' is empty or not found
    if not all_page_no:
        print("Pagination not found on the page.")
        return 1, next

    page_list = [i.text for i in all_page_no[0].find_all('a')]
    
    # Check if 'page_list' is empty or not found
    if not page_list:
        print("Page numbers not found in pagination.")
        return 1, next

    if page_list and any(map(str.isdigit, page_list[-1])):
        return int(page_list[-1]), next
    else:
        next = next + 1
        page_no = int(page_list[-2])
        url = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id=" + sc_id + "&scat=&pageno=" + str(
            page_no) + "&next=" + str(next) + "&durationType=Y&Year=" + str(year) + "&duration=1&news_type="
        return get_page_no(url, sc_id, page_no, next, year)


In [ ]:
def save_company_data(url_ = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?", sc_id=[], page_no=1, next=0, years=[]):
    for company in sc_id:
        df = pd.DataFrame(columns=['company','headline',
                                   'description', 'articleBody'])
        for year in years:
            print('year: ', year)
            print('page_no: ', page_no)
            print('next: ', next)
            
            url = url_ + "sc_id="+company+"&scat=&pageno="+str(page_no)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
            print('url: ', url)
            
            max_page_no, max_next = get_page_no(url, company, page_no, next, year)
            max_next = max_next + 1
            
            for i in range(max_next):
                for j in range((i*10)+1, (i*10)+11):
                    if j <= max_page_no:
                        url_list = []
                        url = url_ + "sc_id="+company+"&scat=&pageno="+str(j)+"&next="+str(i)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
                        request = requests.get(url)
                        soup = BeautifulSoup(request.text, 'html.parser')
                        url_list = get_blog_url(soup)
                        
                        for url in url_list:
                            try:
                                article_dict = get_blog_content(url)
                                print(company)
                                print(article_dict['headline'])
                                print('-------------------------------')
                                
                                article_lst = [[company,
                                                article_dict['headline'],
                                                article_dict['description'],
                                                article_dict['articleBody'],
                                               ]]
                                
                                df = df.append(pd.DataFrame(article_lst, columns=['company', 'headline', 
                                               'description', 'articleBody']), ignore_index = True)
                                
                            except:
                                article_lst = [[company, 'error', 'error', 'error']]
                                df = df.append(pd.DataFrame(article_lst, columns=['company', 'headline', 
                                               'description', 'articleBody']), ignore_index = True)
                                
                                continue
                        else:
                            break
                    
                df.to_csv('company_data/news_scrape_'+company+'.csv', index=False)
                         

In [ ]:
save_company_data(sc_id=['RI'], years = [2023])

In [ ]:
save_company_data(sc_id=['AE01'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['MPS'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['AHE'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['API'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['UTI10'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['BA06'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['BAF'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['BF04'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['BPC'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['BTV'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['BI'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['C'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['CI29'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['DL03'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['DRL'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['EM'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['GI01'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['HCL02'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['HDF01'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['HSL01'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['HHM'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['H'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['HL'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['ICI02'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['ITC'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['IIB'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['IT'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['JVS'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['JFS'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['KMF'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['LI09'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['LT'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['MM'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['MU01'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['NTP'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['NI'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['ONG'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['PGC'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['SLI03'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['SBI'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['SPI'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['TCS'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['TT'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['TEL'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['TIS'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['TM4'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['TI01'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['SI10'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['UTC'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['W'], years = [2023, 2022])

In [ ]:
save_company_data(sc_id=['AE01', 'AHE'], years = [2021])

In [ ]:
print("Success")

In [ ]:
sc_id_values = ['AE01', 'AHE', 'API', 'BA06', 'BAF', 'BF04', 'BI', 'BPC', 'BTV', 'C', 'CI29', 'DL03', 'DRL', 'EM', 'GI01', 'H', 'HCL02', 'HDF01', 'HHM', 'HL', 'HSL01', 'ICI02', 'IIB', 'IT', 'ITC', 'JFS', 'JVS', 'KMF', 'LI09', 'LT', 'MM', 'MPS', 'MU01', 'NI', 'NTP', 'ONG', 'PGC', 'RI', 'SBI', 'SI10', 'SLI03', 'SPI', 'TCS', 'TEL', 'TI01', 'TIS', 'TM4', 'TT', 'UTC', 'UTI10', 'W']

years_of_interest = [2019]

for sc_id in sc_id_values:
    save_company_data(sc_id=[sc_id], years=years_of_interest)

In [2]:
import pandas as pd
import glob
import os

directory_path = r"D:\DEV\Python\Advance_Python_Project\scrap\company_data"
csv_files = glob.glob(f"{directory_path}/*.csv")

combined_data = pd.DataFrame()

for file in csv_files:
    df = pd.read_csv(file)
    combined_data = pd.concat([combined_data, df], ignore_index=True)

output_directory = os.path.dirname(csv_files[0])
output_csv_file = os.path.join(output_directory, "..", "cleaned_data.csv")

combined_data.to_csv(output_csv_file, index=False)
print(f"Combined data saved to {output_csv_file}")


['D:\\DEV\\Python\\Advance_Python_Project\\scrap\\company_data\\news_scrape_RI.csv']
Combined data saved to D:\DEV\Python\Advance_Python_Project\scrap\company_data\..\cleaned_data.csv
